# load module

In [47]:
import subprocess
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.sans-serif'] = 'Arial'
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

In [2]:
finame = '/mnt/Storage/home/wangyiman/source/bySpecies/danRer11_2/ensGene/danRer11_2.ensGene.genePredExt'
tr_gene_pd = pd.read_table(finame, header=None, sep="\t", encoding="utf-8")
tr_gene_pd = tr_gene_pd.iloc[:,[0,11]]
tr_gene_pd.columns = ['transcript_id', 'gene_id']
finame = '/mnt/Storage/home/wangyiman/source/bySpecies/danRer11_2/ensGene/danRer11.ensGeneToGeneSymbol.txt'
gene_symbol_pd = pd.read_table(finame, header=None, sep="\t", encoding="utf-8")
gene_symbol_pd.columns = ['gene_id', 'gene_symbol']
tr_gene_symbol_pd = pd.merge(tr_gene_pd, gene_symbol_pd, on="gene_id")
tr_gene_symbol_pd.head()

,transcript_id,gene_id,gene_symbol
0,ENSDART00000164359.1,ENSDARG00000099104.1,rpl24
1,ENSDART00000158290.1,ENSDARG00000099104.1,rpl24
2,ENSDART00000157701.1,ENSDARG00000099104.1,rpl24
3,ENSDART00000167898.1,ENSDARG00000099104.1,rpl24
4,ENSDART00000166393.1,ENSDARG00000102407.2,cep97


In [3]:
tr_gene_pd_clean = pd.DataFrame(zip(list(zip(*tr_gene_symbol_pd['transcript_id'].str.split('.')))[0], list(zip(*tr_gene_symbol_pd['gene_id'].str.split('.')))[0]))
tr_gene_pd_clean.columns = ['transcript_id_clean', 'gene_id_clean']
tr_gene_pd_clean

,transcript_id_clean,gene_id_clean
0,ENSDART00000164359,ENSDARG00000099104
1,ENSDART00000158290,ENSDARG00000099104
2,ENSDART00000157701,ENSDARG00000099104
3,ENSDART00000167898,ENSDARG00000099104
4,ENSDART00000166393,ENSDARG00000102407
...,...,...
65835,ENSDART00000183313,ENSDARG00000114956
65836,ENSDART00000183313,ENSDARG00000114956
65837,ENSDART00000183313,ENSDARG00000114956
65838,ENSDART00000183313,ENSDARG00000114956


In [4]:
gene_symbol_clean = pd.read_csv('/mnt/Storage/home/wangyiman/source/bySpecies/danRer11_2/ensGene/danRer11.ensGeneToGeneSymbol.clean.txt', header=None, 
                                sep = '\t')
gene_symbol_clean.columns = ['gene_id_clean', 'gene_symbol']
gene_symbol_clean

,gene_id_clean,gene_symbol
0,ENSDARG00000099104,rpl24
1,ENSDARG00000102407,cep97
2,ENSDARG00000102097,nfkbiz
3,ENSDARG00000099319,CU651657.1
4,ENSDARG00000099640,eed
...,...,...
32515,ENSDARG00000109919,CABZ01025162.1
32516,ENSDARG00000116629,CT025651.2
32517,ENSDARG00000114977,CT025651.1
32518,ENSDARG00000102423,FO818711.1


## asign exon

In [5]:
exon_id_ls = pd.DataFrame(zip(*pd.DataFrame(zip(*pd.read_csv('/mnt/Storage/home/wangyiman/source/bySpecies/danRer11_2/ensGene/danRer11_2.ensGene.ExonUniq.gtf', 
                                     sep = '\t', header = 0).iloc[:,8].drop_duplicates().str.split(';'))).T.iloc[:,0].drop_duplicates().str.split('"'))).T.iloc[:,1].tolist()

There is some transcript id like "ENSDART00000139453.3_16.1" in Chr4.

In [6]:
exon_gene_id_dict = {}
exon_gene_id_dict_nonUniq = {}
for exon_id in set(exon_id_ls) :
    trans_ids = [('ENSDART'+x).strip('_') for x in exon_id.split('ENSDART') if x]    
    gene_ids = []
    for trans_id in trans_ids :
        gene_id_series = tr_gene_pd_clean.loc[tr_gene_pd_clean['transcript_id_clean'] == trans_id.split('.')[0]]['gene_id_clean'].tolist()
        if gene_id_series : 
            gene_ids.append(gene_id_series[0])
        else :
            print(exon_id)
        
    if len(set(gene_ids)) == 1 :
        exon_gene_id_dict[exon_id] = gene_ids[0]
    else :
#         print(gene_ids, exon_id)
        exon_gene_id_dict_nonUniq[exon_id] = gene_ids
        

In [7]:
len(exon_gene_id_dict)

320525

In [8]:
len(exon_gene_id_dict_nonUniq)

1890

In [9]:
len(set(exon_id_ls))

322415

In [10]:
320525 + 1890

322415

In [11]:
exon_tr_gene_pd_clean = pd.merge(tr_gene_pd_clean, pd.DataFrame.from_dict(exon_gene_id_dict, orient='index').reset_index(), 
                                  left_on = 'gene_id_clean', right_on = 0).iloc[:,:-1]
exon_tr_gene_pd_clean.columns = ['transcript_id_clean', 'gene_id_clean', 'exon_id']
exon_tr_gene_pd_clean

,transcript_id_clean,gene_id_clean,exon_id
0,ENSDART00000164359,ENSDARG00000099104,ENSDART00000164359.1.3_ENSDART00000167898.1.3
1,ENSDART00000164359,ENSDARG00000099104,ENSDART00000164359.1.4_ENSDART00000167898.1.4
2,ENSDART00000164359,ENSDARG00000099104,ENSDART00000167898.1.1
3,ENSDART00000164359,ENSDARG00000099104,ENSDART00000164359.1.6
4,ENSDART00000164359,ENSDARG00000099104,ENSDART00000158290.1.1_ENSDART00000157701.1.1
...,...,...,...
1130996,ENSDART00000168273,ENSDARG00000104674,ENSDART00000168273.2.5
1130997,ENSDART00000168273,ENSDARG00000104674,ENSDART00000168273.2.7
1130998,ENSDART00000168273,ENSDARG00000104674,ENSDART00000168273.2.4
1130999,ENSDART00000168273,ENSDARG00000104674,ENSDART00000168273.2.8


# merge FPKM TPM readcnt

## FPKM TPM

In [6]:
# add cmd lines to runned.sh
run = False

if run == True :
    os.chdir('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/analysis_embryoExon_tissueExon')
    with open('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/bin/util1_runned_mergeExpr.sh', 'w') as cmd_fi :
        cmd_fi.write('#!/bin/bash')
        for expression_type in ['FPKM', 'TPM'] :
            column = '13' if expression_type == 'FPKM' else '14'
            for finame in ['ovary_earlyEmbryo_sample_exon_readcnt_prepDE_sorted.txt', 'lateEmbryo_tissue_sample_exon_readcnt_prepDE_sorted.txt'] :
                with open(f'{finame}') as samplefi :
                    sample_ls = []
                    firstline = samplefi.readline()
                    gtffi = firstline.strip().split('\t')[1]
                    cmd = 'paste' +f' <(gtf_to_table.py -l 9 -f transcript {gtffi} | grep -v ' + "'#'" +f' | cut -f 10,{column} | sort -k1,1 - )'
                    cmd2 = 'cat <(echo exon_id' + "$'\\t'" + firstline.strip().split("\t")[0]+"$'\\t'"

                    for line in samplefi :
                        gtffi = line.strip().split('\t')[1]
                        cmd += f' <(gtf_to_table.py -l 9 -f transcript {gtffi} | grep -v ' + "'#'" + f' | cut -f 10,{column} | sort -k1,1 - | cut -f 2 ) | paste -'
                        cmd2 += line.strip().split("\t")[0] + "$'\\t'"

                    cmd = cmd[:-10] + f' > ../analysis_embryoExon_tissueExon/{"_".join(finame.split("_")[0:4])}_{expression_type}.tmp'
                    cmd2 = cmd2[:-5] + f') {"_".join(finame.split("_")[0:4])}_{expression_type}.tmp > {"_".join(finame.split("_")[0:4])}_{expression_type}.tsv'
                    cmd_fi.write('\ncd ~/maternal_loading/2.public_data/2_expression_value_ExonUniq_merge_oldNew')
                    cmd_fi.write('\n' + cmd)
                    cmd_fi.write('\ncd ~/maternal_loading/2.public_data/analysis_embryoExon_tissueExon')
                    cmd_fi.write('\n' + cmd2 + '\n')


In [97]:
%%bash
cd ~/maternal_loading/2.public_data
# mkdir 2_expression_value_ExonUniq_merge_oldNew
cd 2_expression_value_ExonUniq_merge_oldNew
ln -f -s ../2_expression_value_ExonUniq/*ExonUniq.* .
ln -f -s ../2_expression_value_trans/tissue_exon/*ExonUniq.* .

In [98]:
import glob
os.chdir('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/2_expression_value_ExonUniq_merge_oldNew')

readcnt_fis = glob.glob('*.read_cnt')
txt_fis = glob.glob('*.txt')
gtf_fis = glob.glob('*.gtf')

readcnt_samples = [x[:-26] for x in readcnt_fis]
txt_samples = [x[8:-30] for x in txt_fis]
gtf_samples = [x[8:-30] for x in gtf_fis]

total_samples = pd.read_csv('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/analysis_embryoExon_tissueExon/ovary_earlyEmbryo_sample_exon_readcnt_prepDE_sorted.txt', 
                            sep = '\t', 
                            header = None).iloc[:,0].tolist() + pd.read_csv('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/analysis_embryoExon_tissueExon/lateEmbryo_tissue_sample_exon_readcnt_prepDE_sorted.txt', 
                                                                           sep = '\t', header = None).iloc[:,0].tolist()

print('.read_cnt', set(total_samples) - set(readcnt_samples))
print('.txt', set(total_samples) - set(txt_samples))
print('.gtf', set(total_samples) - set(gtf_samples))

.read_cnt set()
.txt set()
.gtf set()


In [100]:
%%bash
run=false
cd /mnt/Storage/home/wangyiman/maternal_loading/2.public_data/bin

[ $run == "true" ] && bash util1_runned_mergeExpr.sh


## read count

In [102]:
%%bash
run=false
cd ~/maternal_loading/2.public_data/2_expression_value_ExonUniq_merge_oldNew/
[ $run == "true" ] && sample_list_fi=../analysis_embryoExon_tissueExon/ovary_earlyEmbryo_sample_exon_readcnt_prepDE_sorted.txt && prepDE.py3 -i $sample_list_fi -g ../analysis_embryoExon_tissueExon/ovary_earlyEmbryo_exon_count_matrix.csv -t ../analysis_embryoExon_tissueExon/ovary_earlyEmbryo_exon_count_matrix.csv
[ $run == "true" ] && sample_list_fi=../analysis_embryoExon_tissueExon/lateEmbryo_tissue_sample_exon_readcnt_prepDE_sorted.txt && prepDE.py3 -i $sample_list_fi -g ../analysis_embryoExon_tissueExon/lateEmbryo_tissue_exon_count_matrix.csv -t ../analysis_embryoExon_tissueExon/lateEmbryo_tissue_exon_count_matrix.csv


In [83]:
# run = False
# os.chdir('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/analysis_embryoExon_tissueExon')
# if run == True :
#     overay_early_df = pd.read_table('ovary_earlyEmbryo_exon_count_matrix.csv', header=0, sep=',', index_col=0)
#     late_tissue_df = pd.read_table('lateEmbryo_tissue_exon_count_matrix.csv', header=0, sep=',', index_col=0)
#     overay_early_df_reorder = overay_early_df[expr_3id_dict_T['ovary_earlyEmbryo_FPKM'].index.tolist()]
#     overay_early_df_reorder.to_csv('ovary_earlyEmbryo_exon_count_matrix_reorder.csv', index=True)
#     late_tissue_df_reorder = late_tissue_df[expr_3id_dict_T['lateEmbryo_tissue_FPKM'].index.tolist()]
#     late_tissue_df_reorder.to_csv('lateEmbryo_tissue_exon_count_matrix_reorder.csv', index=True)

# remove batch effect (CombatSeq)

## remove batch effect (readcnt)

util2_remove_batch_effect.R

## readcnt to FPKM

util3_ComBat_readcnt2FPKMandTPM.R